# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("total number of rows: {0}".format(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

total number of rows: 8056


In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("total number of rows with artist name: {0}".format(sum(1 for line in f)))

total number of rows with artist name: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect() # connecting to localhost

#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
session.execute("""
create keyspace if not exists sparkify
with replication = {'class': 'SimpleStrategy', 'replication_factor':1};
""")

#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### added pandas factory for pandas dataframe takes values form the session.row_factory method.

In [9]:
import pandas as pd

def pandas_factory(columns, rows):
    return pd.DataFrame(data=rows, columns=columns)


session.row_factory = pandas_factory
session.default_fetch_size = None

In [10]:
def read_cassandra(query, session):
    query_result = session.execute(query, timeout=None)
    df = query_result._current_rows
    return df

#### Created a table artist_explore_session for queries like 
- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

```
create table artist_explore_session(
sessionId int,
itemInSession int,
artist varchar,
length decimal,
song text,
Primary Key (sessionId, itemInSession)  # composite key with the combination of sessionId and itemInSession
) 
```

In [11]:
# create table in sparkify keyspace to query questions of the data.
session.execute("""
create table artist_explore_session(
sessionId int,
itemInSession int,
artist varchar,
length decimal,
song text,
Primary Key (sessionId, itemInSession)
) 
""")

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
            INSERT INTO artist_explore_session(sessionid, iteminsession, artist,  length, song)
            values (%s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

In [13]:
query_1 = " select artist, song, length from artist_explore_session where sessionid=338 and itemInSession=4 "                 
query1_df = read_cassandra(query=query_1, session=session)
query1_df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


#### Created a table song_playlist_session for queries like 
- Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```
create table song_playlist_session(
userId int,
sessionId int,
itemInSession int,
artist varchar,
firstName varchar,
gender varchar,
lastName varchar,
length decimal,
level varchar,
location text,
song text,
Primary Key ((userId,sessionId), itemInSession)  
) with clustering order by (itemInSession ASC)
) # composite key with the combination of userId, sessionId and with clustering key itemInSession in ascending order. 
```

In [14]:
# create table in sparkify keyspace to query questions of the data.
session.execute("""
create table song_playlist_session(
userId int,
sessionId int,
itemInSession int,
artist varchar,
firstName varchar,
gender varchar,
lastName varchar,
length decimal,
level varchar,
location text,
song text,
Primary Key ((userId,sessionId), itemInSession) 
) with clustering order by (itemInSession ASC)
""")

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO song_playlist_session(userId,sessionId,itemInSession,artist,firstName,gender,lastName,length,level,location,song)
        values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[1],line[2],line[4],float(line[5]),line[6],line[7],line[9]))

In [16]:
query_2 = " select artist, song, firstName as user_firstName, lastName as user_lastName from song_playlist_session where userId=10 and sessionid=182"
query2_df = read_cassandra(query=query_2, session=session)
query2_df

,artist,song,user_firstname,user_lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Created a table user_activity for queries like 
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```
create table user_activity(
song text,
userId int,
firstName varchar,
gender varchar,
lastName varchar,
length decimal,
level varchar,
location text,
Primary Key (song, userId)
) # composite key with the combination of song and userId
```

In [17]:
# create table in sparkify keyspace to query questions of the data.
session.execute("""
create table user_activity(
song text,
userId int,
firstName varchar,
gender varchar,
lastName varchar,
length decimal,
level varchar,
location text,
Primary Key (song, userId)
)
""")

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO user_activity(song,userId,firstName,gender,lastName,length,level,location)
        values (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (line[9], int(line[10]),line[1], line[2], line[4], float(line[5]), line[6], line[7]))

In [19]:
query_3 = "select firstName, lastName from user_activity where song = 'All Hands Against His Own'"        
query3_df = read_cassandra(query=query_3, session=session)
query3_df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [20]:
## TO-DO: Drop the table before closing out the sessions
session.execute("drop table artist_explore_session")
session.execute("drop table song_playlist_session")
session.execute("drop table user_activity")
session.execute("drop keyspace sparkify")

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()